In [16]:
from itertools import product, combinations
import matplotlib.pyplot as plt
import numpy as np
from src import k_tree
from src.utils import data as dt
import torch
from src.k_tree import Ktree
from src.utils.data import loadData, loadData_3d
from src.metrics import Linf_3d

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data, _ = loadData_3d(10000, 100)

if type(data) is not torch.Tensor:
    data = torch.tensor(data).to(device)
n = 100  # number of points
data = data[torch.randperm(data.size(0))][:n]
# normalize data
x_min = data[:, 0].min().item()
x_max = data[:, 0].max().item()
y_min = data[:, 1].min().item()
y_max = data[:, 1].max().item()
z_min = data[:, 2].min().item()
z_max = data[:, 2].max().item()
l_max = data[:, 3].max().item()
w_max = data[:, 4].max().item()
h_max = data[:, 5].max().item()
bbox = torch.tensor(
    [
        x_min - l_max,
        x_max + l_max,
        y_min - w_max,
        y_max + w_max,
        z_min - h_max,
        z_max + h_max,
    ]
)
print(f"bbox: {bbox}")
# normalize
data[:, 0] = (data[:, 0] - bbox[0]) / (bbox[1] - bbox[0])
data[:, 1] = (data[:, 1] - bbox[2]) / (bbox[3] - bbox[2])
data[:, 2] = (data[:, 2] - bbox[4]) / (bbox[5] - bbox[4])
max_xyz = max(bbox[1] - bbox[0], bbox[3] - bbox[2], bbox[5] - bbox[4])
data[:, 3] = data[:, 3] / max_xyz
data[:, 4] = data[:, 4] / max_xyz
data[:, 5] = data[:, 5] / max_xyz
x_min = data[:, 0].min().item()
x_max = data[:, 0].max().item()
y_min = data[:, 1].min().item()
y_max = data[:, 1].max().item()
z_min = data[:, 2].min().item()
z_max = data[:, 2].max().item()
l_max = data[:, 3].max().item()
w_max = data[:, 4].max().item()
h_max = data[:, 5].max().item()
bbox = torch.tensor(
    [
        x_min - l_max,
        x_max + l_max,
        y_min - w_max,
        y_max + w_max,
        z_min - h_max,
        z_max + h_max,
    ]
)
print(f"bbox: {bbox}")

Loading data...
Data loaded.
bbox: tensor([ 1.0551e+01,  4.0005e+03, -3.1377e+00,  3.9159e+03,  6.9080e+01,
         3.8730e+03])
bbox: tensor([8.8590e-11, 1.0000e+00, 1.8134e-05, 9.9998e-01, 4.9036e-05, 9.9995e-01])


In [18]:
import torch

# from src.k_tree_poly import Ktree as Ktree_pls
from src.k_tree_poly_copy import Ktree as Ktree
from src.utils.objects.squares import loadData as loadSquares
from src.utils import plot_tools as pt
from src.utils import accuracy as acc

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dim = 3  # space dimension

k = 3  # number of centroids to generate in the Clustering model
clustering_args = {
    "epochs": 10,  # number of epochs
    "pre_processing": 10,
    "number_of_centroids": k,  # number of centroids to generate in the Clustering model
    "dimension": dim,  # space dimension
    "object_id": "cuboids",  # object id
}
# n = 50
n = 30
un_args = {
    "N": n,  # number of points to sample
    "M": n**3 - 1,  # number of points to return
    "epsilon": 0.2,  # the epsilon ball. Increase this to get more points (as var increases)
    # .15
}
critic_args = {
    "optimizer_lr": 5e-3,  # optimiser learning rate
    "epochs": 2000,  # number of epochs
    "width": 300,  # width of the model's linear layers
    "depth": 5,  # depth of the model's linear layers
}

# threshold = 100 - 1
threshold = 3 * k
# threshold = 998
threshold = 98

# Initialise the k-tree structure.
from src.metrics import Linf_3d

# pass data to the k-tree
distance_function = Linf_3d
ktree = Ktree(
    threshold,
    data,
    distance_function,
    clustering_args,
    un_args,
    critic_args,
    device,
    dim,
)
# pass data to device
# data = data.to(device)
print("Starting to create the tree...")
print("=" * 20)
# ktree.create_tree(save_path_prefix="./models/line_segments/2d/5k/", plot=False)
ktree.create_tree()

Starting to create the tree...

Creating critic for node 0 that has 100 data, which is more than the threshold 98.
Bounding box for node 0: [[8.859006410399939e-11, 1.0000000321737281], [1.8133650473523173e-05, 0.999981841201762], [4.90358017948458e-05, 0.99995097158907]]
Creating clustering for node 0 with 3 centroids.
Starting centroid 2
Starting centroid 3
Initial divergence: 31.99117660522461
Starting iteration 1
Centroids:  tensor([[0.0642, 0.8012, 0.7006],
        [0.6436, 0.6934, 0.5137],
        [0.3433, 0.6156, 0.0964]], device='cuda:0')
Iteration 1, divergence: 0.31991177797317505
Starting iteration 2
Centroids:  tensor([[0.2049, 0.5984, 0.7234],
        [0.6696, 0.6262, 0.5496],
        [0.3948, 0.3987, 0.2669]], device='cuda:0')
Iteration 2, divergence: 0.2769835293292999
Starting iteration 3
Centroids:  tensor([[0.2345, 0.5986, 0.7143],
        [0.7049, 0.7090, 0.5082],
        [0.4341, 0.3449, 0.2823]], device='cuda:0')
Iteration 3, divergence: 0.2702225148677826
Starting

In [19]:
print(f"Tree node number is {ktree.number_of_nodes}")

leaves = ktree.get_leaves()

height = max([len(leaf.index) for leaf in leaves])
print(f"Tree height is {height}.")

leaf_sizes = [len(leaf.data) for leaf in leaves]
print(f"Created {len(leaves)} leaves with sizes")
print(leaf_sizes)

Tree node number is 4
Tree height is 2.
Created 3 leaves with sizes
[39, 31, 30]


In [20]:
acc.random_queries(ktree, 100, 1, 1)

The number of queries per layer are:
[100.]
The percentage of correct predictions per layer is:
[98.]
The mean percentage of correct predictions per layer is:
[98.]
